In [32]:
import os
import sys 
import numpy as np 
import tensorflow as tf
import operator
batch_size =50 

def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder and the prediction operation. """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path)
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    
def load_validation_data(model, testname):
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...")
    folder = 'time_series_data/' + testname#one_half_sec/samefiles'#LT_  Data_total_time/test_data'#Data_total_time/test_data'#GTTP Data_total_time/test_data'#Data_total_time/test_data'#Data_total_time/test_data'#Data_total_time/test_data'#Half_test'#Data_total_time/test_data'#two_sec'#'
    if model==1:
        testX  =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_true.npy' ) 
        testY  =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_true.npy' )    
    elif model ==2:
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_pred.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_pred.npy'  )  
    elif model==3:
        testX =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testX_no.npy' ) 
        testY =np.load('E:/identification_data/01_PSSE_datasets/Dataset/saved_data_jupyter/'+folder+'/testY_no.npy' )  
    if testX.shape[1]!= 214:
        testX = np.transpose(testX,[0,2,1])
        testX = np.expand_dims(testX,3) 
    batch_size =np.shape(testX)[0]    
    print (batch_size)
    return (testX,testY )    
def validate_model(session, val_data, x, y,  phase_train,prob): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...")
    #y = tf.placeholder(tf.int32, [None, 3]) 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y  = val_data
    test_correct = session.run(correct,feed_dict ={x: val_x,  y:val_y, phase_train: True})
    test_acc = session.run(acc,feed_dict ={x: val_x, y:val_y, phase_train: True})
    return ( test_acc), test_correct 
'''def validate_model(session, val_data, x,  phase_train,predict_op): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...")
    y = tf.placeholder(tf.int32, [None, 3]) 
    correct = tf.equal(predict_op,tf.argmax(y, 1))
# Compute total number of correct answers.
    total_correct =tf.reduce_sum(tf.cast(correct, tf.float32))  

  # Validate the model.
    val_x, val_y  = val_data
    num_iters = int(np.floor(val_x.shape[0] / batch_size))
    print (num_iters) 
    all_correct = 0
    for i in range(0, num_iters):
        start_index = i * batch_size
        end_index = start_index + batch_size
        val_batch = val_x[start_index:end_index] 
        label_batch = val_y[start_index:end_index]
        print("Validating batch %d of %d..." % (i + 1, num_iters))
        all_correct +=session.run(total_correct, feed_dict={x: val_batch, y :label_batch , phase_train : True  }) 
        correct_results=session.run(correct, feed_dict={x: val_batch,  y :label_batch , phase_train : True  }) 
    return all_correct / val_x.shape[0],correct_results'''
def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder, mask placeholder, and the prediction operation. """ 
    valid_nodes = tf.get_collection_ref("validation_nodes")
    #print (valid_nodes[3])
    '''if len(valid_nodes) != 4:
        print("ERROR: Expected 4 items in validation_nodes, got %d." %(len(valid_nodes)))
        sys.exit(1) '''
    # Figure out which is which.
    x = valid_nodes[0] 
    y = valid_nodes[1] 
    phase_train =valid_nodes[2]
    prob =  valid_nodes[3]
    global batch_size 
    #batch_size = int(x.get_shape()[0])  
    # Predict op should also yield integers.
    #predict = tf.cast(predict_op, "int32") 
    # Check the shape of the prediction output.
    #p_shape = predict.get_shape()
    #Commented these out because there could be squeezes in the code earlier
    """
  print p_shape
  if len(p_shape) > 2:
    print("ERROR: Expected prediction of shape (<X>, 1), got shape of %s." % \
          (str(p_shape)))
    sys.exit(1)
  if len(p_shape) == 2:
    if p_shape[1] != 1:
      print("ERROR: Expected prediction of shape (<X>, 1), got shape of %s." % \
            (str(p_shape)))
      sys.exit(1)

    # We need to contract it into a vector.
    predict = predict[:, 0]""" 
    return (x,y,  phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1) 
    
    acc = []
    zero = np.where(label_y==0) 
    correctl=[correct_results[i] for i in zero]
    accuracy_zero= np.mean(correctl,dtype=np.float32)
    acc.append(accuracy_zero)

    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one] 
    accuracy_one= np.mean(correctg,dtype=np.float32)
    acc.append(accuracy_one)

    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two] 
    accuracy_two= np.mean(correctt,dtype=np.float32) 
    acc.append(accuracy_two) 
    #print('The accuracy of the linetrip events is %.2f \n'  %(100*accuracy_zero))
    #print('The accuracy of the generator trip events is %.2f \n' %(100-100*accuracy_one))
    #print('The accuracy of the line faults events is %.2f \n' %(100-100*accuracy_two) )
    return acc, correctl,correctg,correctt
def reset():
    tf.reset_default_graph()

num_save=1
model = 3
testname = 'one_sec'
if testname == 'one_sec':
    used = [768, 4, 261, 262, 11, 12, 791, 26, 177, 951, 957, 192, 964, 716, 717, 980, 985, 219, 733, 227, 229, 741, 1127, 250, 1135, 752, 753, 1139, 1140, 756, 757, 246, 1146, 1149]
elif testname == 'two_sec':
    used = [385, 646, 392, 650, 656, 400, 401, 660, 407, 410, 155, 420, 423, 427, 450, 581, 1225, 591, 1237, 1238, 342, 991, 1248, 1249, 352, 1004, 627, 632, 381]
elif testname == 'half_sec':
    used = [282, 288, 1058, 1063, 1070, 1198, 690, 1203, 1076, 308, 1079, 1208, 315, 700, 64, 708, 709, 1094, 1222, 1098, 76, 79, 1107, 86, 1114, 866, 879, 113, 888, 890]
elif testname == 'one_half_sec':
    used = [1024, 1159, 1162, 1044, 534, 1046, 536, 1175, 540, 1054, 159, 928, 1185, 548, 549, 933, 40, 941, 46, 561, 52, 821, 949, 950, 572, 834, 463, 854, 479, 481, 501, 510, 1023]  
#save_path = 'E:/05_research/04_deep_learning_model/Save_parameters/best_model/model2' 
names ='fully_best/modelC_fully02'# 'modelC_ratio18_set01'##'ratio_80_best/modelC_ratio80_set02'# #'modelC_fully02'['modelC_fully2']#'modelC_ratio50_set01'['modelC_ratio100_set03']#['modelC_ratio80_set02']# ['modelC_ratio24_set01']#['modelC_small01']
with tf.Session() as session:  
    save_path = 'E:/05_research/04_deep_learning_model/Save_parameters/modelC/'+ names 
    x, y,phase_train,prob  = load_model(session, save_path)
    val_data = load_validation_data(model, testname) # the predicted mode
    total = [i for i in range(np.shape(val_data[0])[0])]
    remain = list(set(total) - set(used)); 
    accuracy[0,i] ,correct_results= validate_model(session, val_data, x,y, phase_train,prob)
    remain_results = np.array([correct_results[i] for i in remain] )
    labels = [val_data[1][i] for i in remain]
    #print ("Overall validation accuracy is %f \n" %(accuracy))
    acc,correct_l,correct_g,correct_t= each_perform(remain_results,labels )  
    session.close()     
id_True =  np.nonzero(remain_results==True) 
overall_acc = (len(id_True[0])/len(remain_results))   
  

print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t))
print ('Overall:', 100*overall_acc) 

Loading model from file 'E:/05_research/04_deep_learning_model/Save_parameters/modelC/fully_best/modelC_fully02'...
INFO:tensorflow:Restoring parameters from E:/05_research/04_deep_learning_model/Save_parameters/modelC/fully_best/modelC_fully02
Loading validation data...
322
Validating model...
LT: 94.84536082474226
GT: 83.16831683168317
TP: 78.37837837837837
Overall: 85.11326860841424


In [11]:
id_false =  np.nonzero(remain_results==False) 
id_false

(array([  0,   1,   2,   6,   7,  10,  16,  17,  22,  23,  28,  29,  34,
         35,  36,  41,  42,  47,  48,  49,  54,  55,  60,  61,  62,  67,
         68,  86,  87,  88,  91,  93,  94,  99, 100, 101, 105, 107, 108,
        113, 114, 115, 116, 120, 121, 126, 127, 128, 129, 133, 134, 139,
        140, 141, 142, 146, 147, 148, 152, 153, 154, 159, 160, 165, 166,
        167, 168, 171, 172, 173, 183, 190, 191, 192, 193, 194, 195, 196,
        198, 199, 200, 209, 210, 212, 219, 221, 222, 223, 224, 225, 227,
        228, 229, 230, 231, 235, 236, 237, 240, 241, 242, 245, 246, 247,
        249, 250, 259, 261, 262], dtype=int64),)

In [7]:
id_f = [    0,    1,    2,    6,    7,   10,   16,   17,   22,   23,   28,
          29,   34,   35,   40,   45,   46,   51,   52,   57,   58,   63,
          64,   79,   80,   81,   84,   86,   87,   92,   93,   94,   98,
         100,  101,  105,  106,  107,  108,  112,  113,  118,  119,  120,
         121,  125,  126,  131,  132,  133,  134,  138,  139,  140,  143,
         144,  145,  149,  150,  155,  156,  157,  158,  161,  162,  163,
         173,  180,  181,  182,  183,  184,  185,  186,  188,  189,  190,
         199,  200,  202,  209,  211,  212,  213,  214,  215,  217,  218,
         219,  220,  221,  225,  226,  227,  230,  231,  232,  235,  236,
         237,  239,  240,  249,  251,  252,  314,  315,  316,  317,  318,
         319,  321,  322,  323,  324,  341,  342,  343,  344,  345,  346,
         347,  349,  351,  359,  366,  367,  368,  369,  370,  374,  383,
         389,  390,  391,  392,  396,  408,  417,  418,  419,  420,  421,
         422,  424,  425,  426,  427,  428,  437,  443,  444,  445,  446,
         447,  448,  449,  451,  453,  454,  461,  462,  469,  470,  471,
         472,  473,  474,  475,  478,  479,  481,  495,  496,  499,  511,
         512,  534,  577,  605,  606,  607,  608,  609,  610,  611,  612,
         613,  614,  615,  616,  617,  618,  619,  620,  621,  622,  623,
         624,  625,  626,  627,  628,  629,  630,  631,  632,  633,  634,
         635,  636,  637,  638,  639,  640,  641,  642,  643,  644,  645,
         646,  659,  718,  752,  784,  874,  875,  880,  885,  886,  887,
         888,  891,  892,  895,  896,  897,  898,  901,  902,  906,  912,
         917,  918,  919,  920,  923,  924,  925,  926,  927,  928,  929,
         930,  934,  935,  936,  939,  940,  941,  942,  946,  947,  950,
         952,  956,  958,  960, 1016, 1027, 1030, 1032, 1036, 1037, 1039,
        1043, 1045, 1046, 1048, 1049, 1050, 1051, 1052, 1054, 1055, 1056,
        1058, 1059, 1060, 1061, 1062, 1063, 1066, 1067, 1069, 1070, 1071,
        1072, 1073, 1074, 1077, 1078, 1081, 1082, 1083, 1087, 1088, 1091,
        1092, 1093, 1094, 1095, 1096, 1097, 1098, 1100, 1101, 1102, 1104,
        1109, 1114, 1115, 1117, 1119, 1120, 1121, 1122, 1123, 1124, 1125,
        1126, 1127, 1130, 1131, 1132, 1133, 1134, 1135]

In [8]:
if testname == 'total':
    s1 = 14; s2 = 14*3-4; s3 = 14*3 -8; s4 =12; s5 = 9; s6= 19;
    id1 = [i for i in id_f if i < 169-s1]
    id2 = [i for i in id_f if i >= 169-s1 and i < (169+404-s2)]
    id3 = [i for i in id_f if i >= (169+404-s2) and i < (169+404+378-s3)]
    id4 = [i for i in id_f if i >= (169+404+378-s3) and i < (169+404+378+107-s4)]
    id5 = [i for i in id_f if i >= (169+404+378+107-s4) and i < (169+404+378+107+58-s5)]
    id6 = [i for i in id_f if i >= (169+404+378+107+58-s5) and i < 1262-s6]
    acc1 = (1-len(id1)/(169-s1))*100 ; print ( acc1)
    acc2 = (1-len(id2)/(404-s2))*100 ; print ( acc2)
    acc3 = (1-len(id3)/(378-s3))*100 ; print ( acc3)
    acc4 = (1-len(id4)/(107-s4))*100 ; print ( acc4)
    acc5 = (1-len(id5)/(58-s5))*100 ; print ( acc5)
    acc6 = (1-len(id6)/(146-s6))*100 ; print ( acc6)

61.935483870967744
67.4863387978142
81.3953488372093
63.1578947368421
14.28571428571429
85.03937007874016


In [6]:
if testname == 'total90':
    s1 = 14; s2 = 14*3-4; s3 = 14*3 -8; s4 =12; s5 = 9; s6= 19;
    id1 = [i for i in id_f if i < 169-s1]
    id2 = [i for i in id_f if i >= 169-s1 and i < (169+404-s2)]
    id3 = [i for i in id_f if i >= (169+404-s2) and i < (169+404+378-s3)]
    id4 = [i for i in id_f if i >= (169+404+378-s3) and i < (169+404+378+107-s4)]
    id5 = [i for i in id_f if i >= (169+404+378+107-s4) and i < (169+404+378+107+58-s5)]
    id6 = [i for i in id_f if i >= (169+404+378+107+58-s5) and i < 1262-s6]
    acc1 = (1-len(id1)/(169-s1))*100 ; print ( acc1)
    acc2 = (1-len(id2)/(404-s2))*100 ; print ( acc2)
    acc3 = (1-len(id3)/(378-s3))*100 ; print ( acc3)
    acc4 = (1-len(id4)/(107-s4))*100 ; print ( acc4)
    acc5 = (1-len(id5)/(58-s5))*100 ; print ( acc5)
    acc6 = (1-len(id6)/(146-s6))*100 ; print ( acc6)
if testname == 'total':    
    id1 = [i for i in id_f if i < 169]
    id2 = [i for i in id_f if i >= 169 and i < (169+404)]
    id3 = [i for i in id_f if i >= (169+404) and i < (169+404+378)]
    id4 = [i for i in id_f if i >= (169+404+378) and i < (169+404+378+107)]
    id5 = [i for i in id_f if i >= (169+404+378+107) and i < (169+404+378+107+58)]
    id6 = [i for i in id_f if i >= (169+404+378+107+58) and i < 1262]
    acc1 = (1-len(id1)/169)*100 ; print ( acc1)
    acc2 = (1-len(id2)/404)*100 ; print ( acc2)
    acc3 = (1-len(id3)/378)*100 ; print ( acc3)
    acc4 = (1-len(id4)/107)*100 ; print ( acc4)
    acc5 = (1-len(id5)/58)*100 ; print ( acc5)
    acc6 = (1-len(id6)/146)*100 ; print ( acc6)

63.87096774193548
65.30054644808743
80.52325581395348
63.1578947368421
26.530612244897956
85.8267716535433
